In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transform
from torchvision.transforms import ToTensor
import torchvision.datasets as datasets
from torch.utils.data import Dataset,DataLoader


class CNNClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Conv2d(1, 64, 3),
                                 nn.ReLU(),
                                 nn.MaxPool2d((2, 2), stride=2),
                                 nn.Conv2d(64, 128, 3),
                                 nn.ReLU(),
                                 nn.MaxPool2d((2, 2), stride=2),
                                 nn.Conv2d(128, 64, 3),
                                 nn.ReLU(),
                                 nn.MaxPool2d((2, 2), stride=2),
                                 )
        self.classification_head = nn.Sequential(nn.Linear(64, 20, bias=True),
                                                 nn.ReLU(),
                                                 nn.Linear(20, 10, bias=True), )

    def forward(self, x):
        features = self.net(x)
        return self.classification_head(features.view(batch_size, -1))

batch_size = 50

mnist_testset = datasets.FashionMNIST(root='./data', train=False,download=True, transform = ToTensor())
test_loader = DataLoader(mnist_testset, batch_size=batch_size, shuffle=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNClassifier()
model = torch.load("./model.pt")
model.to(device)

print("Model's state_dict:")
for param_tensor in model.state_dict().keys():
    print(param_tensor, "\t",model.state_dict()[param_tensor].size())
print()

model.eval()
correct = 0
total = 0
for i, vdata in enumerate(test_loader):
    tinputs, tlabels = vdata
    tinputs = tinputs.to(device)
    tlabels = tlabels.to(device)
    toutputs = model(tinputs)
    #Select the predicted class label which has the
    # highest value in the output layer
    _, predicted = torch.max(toutputs, 1)
    print("True label:{}".format(tlabels))
    print('Predicted: {}'.format(predicted))
    # Total number of labels
    total += tlabels.size(0)
    # Total correct predictions
    correct += (predicted == tlabels).sum()
accuracy = 100.0 * correct / total
print("The overall accuracy is {}".format(accuracy))

Model's state_dict:
net.0.weight 	 torch.Size([64, 1, 3, 3])
net.0.bias 	 torch.Size([64])
net.3.weight 	 torch.Size([128, 64, 3, 3])
net.3.bias 	 torch.Size([128])
net.6.weight 	 torch.Size([64, 128, 3, 3])
net.6.bias 	 torch.Size([64])
classification_head.0.weight 	 torch.Size([20, 64])
classification_head.0.bias 	 torch.Size([20])
classification_head.2.weight 	 torch.Size([10, 20])
classification_head.2.bias 	 torch.Size([10])



/opt/anaconda3/envs/pytorch_env/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1702400440653/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


True label:tensor([9, 2, 1, 1, 6, 1, 4, 6, 5, 7, 4, 5, 7, 3, 4, 1, 2, 4, 8, 0, 2, 5, 7, 9,
        1, 4, 6, 0, 9, 3, 8, 8, 3, 3, 8, 0, 7, 5, 7, 9, 6, 1, 3, 7, 6, 7, 2, 1,
        2, 2], device='cuda:0')
Predicted: tensor([0, 6, 0, 3, 0, 0, 0, 8, 5, 4, 6, 6, 6, 6, 6, 3, 0, 0, 0, 8, 0, 6, 6, 6,
        6, 6, 6, 6, 0, 8, 0, 6, 1, 8, 6, 8, 5, 4, 4, 2, 8, 8, 6, 2, 6, 3, 6, 8,
        8, 0], device='cuda:0')
True label:tensor([4, 4, 5, 8, 2, 2, 8, 4, 8, 0, 7, 7, 8, 5, 1, 1, 2, 3, 9, 8, 7, 0, 2, 6,
        2, 3, 1, 2, 8, 4, 1, 8, 5, 9, 5, 0, 3, 2, 0, 6, 5, 3, 6, 7, 1, 8, 0, 1,
        4, 2], device='cuda:0')
Predicted: tensor([0, 6, 5, 0, 8, 0, 0, 6, 0, 2, 4, 6, 6, 0, 0, 3, 8, 8, 5, 0, 4, 3, 0, 6,
        6, 8, 8, 6, 6, 0, 8, 0, 5, 0, 6, 8, 6, 6, 8, 6, 4, 6, 6, 4, 0, 0, 6, 8,
        6, 6], device='cuda:0')
True label:tensor([3, 6, 7, 2, 7, 8, 5, 9, 9, 4, 2, 5, 7, 0, 5, 2, 8, 6, 7, 8, 0, 0, 9, 9,
        3, 0, 8, 4, 1, 5, 4, 1, 9, 1, 8, 6, 2, 1, 2, 5, 1, 0, 0, 0, 1, 6, 1, 6,
        2, 2], de

In [35]:
import PIL.Image
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import glob
import torchvision
from torchvision import transforms
from torchvision.models import alexnet
import torchvision.datasets as datasets
import pandas as pd

model = torchvision.models.alexnet(pretrained=True)

class MyDataset(Dataset):
    def __init__(self, transform, str="train"):
        self.img_path = "D:\\Datasets\\cats_and_dogs_filtered\\" + str + "\\" 
        file_list = glob.glob(self.img_path + "*")
        self.data = []
        for class_path in file_list:
            class_name = class_path.split("\\")[-1]
            for img_path in glob.glob(class_path + "\\*.jpg"):
                self.data.append([img_path, class_name])
                #print(img_path, class_name)
        self.class_map = {"dogs": 0, "cats": 1}
        self.transform = transform
        self.label = [self.class_map[class_name] for _, class_name in self.data]
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_path, class_name = self.data[idx]
        img = PIL.Image.open(img_path)
        img = self.transform(img)
        label = self.class_map[class_name]
        return img, label

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = MyDataset(preprocess, str="train")
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, drop_last=True)
test_dataset = MyDataset(preprocess, str="validation")
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True, drop_last=True)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 2)
device = "cpu"

model.eval()
correct = 0
total = 0
for i, (tinputs, tlabels) in enumerate(test_dataloader):
    toutputs = model(tinputs)
    _, predicted = torch.max(toutputs, 1)
    total += tlabels.size(0)
    correct += (predicted == tlabels).sum().item()
accuracy = 100.0 * correct / total
print("Correct:",correct,"Total:",total)
print("The overall accuracy is {:.2f}%".format(accuracy))

Correct: 451 Total: 1000
The overall accuracy is 45.10%


In [5]:
import PIL.Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import glob
import torchvision
import torchvision.transforms as transform
from torchvision.models import AlexNet_Weights
from torchvision.transforms import ToTensor
import torchvision.datasets as datasets

class CNNClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Conv2d(1, 64, 3),
                                 nn.ReLU(),
                                 nn.MaxPool2d((2, 2), stride=2),
                                 nn.Conv2d(64, 128, 3),
                                 nn.ReLU(),
                                 nn.MaxPool2d((2, 2), stride=2),
                                 nn.Conv2d(128, 64, 3),
                                 nn.ReLU(),
                                 nn.MaxPool2d((2, 2), stride=2),
                                 )
        self.classification_head = nn.Sequential(nn.Linear(64, 20, bias=True),
                                                 nn.ReLU(),
                                                 nn.Linear(20, 10, bias=True), )

    def forward(self, x):
        features = self.net(x)
        return self.classification_head(features.view(batch_size, -1))


batch_size=4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNClassifier().to(device)

check_point = torch.load("./checkpoints/checkpoint.pt")
model.load_state_dict(check_point["model_state"])

mnist_trainset = datasets.FashionMNIST(root='./data', train=False,download=True, transform = ToTensor())
train_loader = DataLoader(mnist_trainset, batch_size=batch_size, shuffle=False)
mnist_testset = datasets.FashionMNIST(root='./data', train=False,download=True, transform = ToTensor())
test_loader = DataLoader(mnist_testset, batch_size=batch_size, shuffle=False)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.001)
optimizer.load_state_dict(check_point["optimizer_state"])
loss = check_point["last_loss"]
epoch = check_point["last_epoch"]

NEW_EPOCHS = 5
for epoch in range(NEW_EPOCHS):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

correct, total = 0, 0
for i, vdata in enumerate(test_loader):
    tinputs, tlabels = vdata[0].to(device), vdata[1].to(device)
    toutputs = model(tinputs)
    _, predicted = torch.max(toutputs, 1)
    total += tlabels.size(0)
    correct += (predicted == tlabels).sum()
print(f"Correct = {correct}, Total = {total}")
accuracy = 100.0 * correct / total
print("The overall accuracy is {}".format(accuracy))

[1,   100] loss: 2.024
[1,   200] loss: 1.506
[1,   300] loss: 1.244
[1,   400] loss: 1.124
[1,   500] loss: 1.055
[1,   600] loss: 0.989
[1,   700] loss: 0.972
[1,   800] loss: 1.055
[1,   900] loss: 0.936
[1,  1000] loss: 1.012
[1,  1100] loss: 0.959
[1,  1200] loss: 0.753
[1,  1300] loss: 0.894
[1,  1400] loss: 0.844
[1,  1500] loss: 0.784
[1,  1600] loss: 0.804
[1,  1700] loss: 0.805
[1,  1800] loss: 0.766
[1,  1900] loss: 0.771
[1,  2000] loss: 0.770
[1,  2100] loss: 0.752
[1,  2200] loss: 0.735
[1,  2300] loss: 0.739
[1,  2400] loss: 0.787
[1,  2500] loss: 0.690
[2,   100] loss: 0.712
[2,   200] loss: 0.796
[2,   300] loss: 0.731
[2,   400] loss: 0.689
[2,   500] loss: 0.687
[2,   600] loss: 0.638
[2,   700] loss: 0.667
[2,   800] loss: 0.813
[2,   900] loss: 0.732
[2,  1000] loss: 0.775
[2,  1100] loss: 0.706
[2,  1200] loss: 0.568
[2,  1300] loss: 0.700
[2,  1400] loss: 0.694
[2,  1500] loss: 0.621
[2,  1600] loss: 0.657
[2,  1700] loss: 0.645
[2,  1800] loss: 0.611
[2,  1900] 